# 🚌 Bus Label Detection - YOLOv11 Training & CoreML Export

This notebook trains a YOLOv11 model to detect bus labels and exports it to CoreML format for iOS integration.

## Workflow:
1. Install dependencies (Ultralytics YOLO)
2. Download training dataset from Roboflow
3. Train YOLOv11 model on bus label data
4. Export trained model to CoreML format (.mlpackage)
5. Save model to Google Drive for download

## Output:
- **PyTorch model**: `best.pt` - Original trained model
- **CoreML model**: `yolov11n.mlpackage` - Ready for iOS/Swift integration

## Requirements:
- Google Colab with GPU runtime (T4 recommended)
- Roboflow API key (already configured)
- Google Drive for storing exported models


In [ ]:

import os
HOME = os.getcwd()
print(HOME)


In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image


In [ ]:
%cd {HOME}
# !yolo task=detect mode=predict model=yolo12s.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:

!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet
# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(3)
# dataset = version.download("yolov12")

# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(3)
# dataset = version.download("yolov11")

# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(6)
# dataset = version.download("yolov11")

from roboflow import Roboflow
rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
version = project.version(7)
dataset = version.download("yolov11")


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11n.pt data=/content/datasets/aisee-test-7/data.yaml epochs=100 imgsz=640 plots=True

In [ ]:
from IPython.display import FileLink, HTML
p = "/content/runs/detect/train/weights/best.pt"
FileLink(p)  # clickable link in the output


In [ ]:
# Save the PyTorch model to Google Drive
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Check if model exists before copying
model_path = "/content/runs/detect/train/weights/best.pt"
if os.path.exists(model_path):
    !cp "{model_path}" "/content/drive/MyDrive/best.pt"
    print("✅ PyTorch model saved to Google Drive")
else:
    print("❌ Model file not found. Make sure training completed successfully.")
    print(f"   Expected location: {model_path}")

In [ ]:
# Export the trained model to CoreML format for iOS integration
# This creates a .mlpackage file that can be used in Xcode

from ultralytics import YOLO
import os

model_path = '/content/runs/detect/train/weights/best.pt'

# Check if model exists before exporting
if os.path.exists(model_path):
    # Load the best trained model
    model = YOLO(model_path)
    
    # Export to CoreML format
    # nms=True includes Non-Maximum Suppression in the model for better iOS performance
    # imgsz=640 matches the training image size
    model.export(format='coreml', nms=True, imgsz=640)
    
    print("✅ Model exported to CoreML format successfully!")
    print("📦 The exported model will be in the same directory as best.pt")
    print("Look for: best.mlpackage or best_nms.mlpackage")
else:
    print("❌ Model file not found. Make sure training completed successfully.")
    print(f"   Expected location: {model_path}")


In [ ]:
# List the exported CoreML model files
import os

weights_dir = "/content/runs/detect/train/weights/"

if os.path.exists(weights_dir):
    print("Files in weights directory:")
    mlmodels_found = False
    for file in os.listdir(weights_dir):
        if file.endswith('.mlpackage') or file.endswith('.mlmodel'):
            full_path = os.path.join(weights_dir, file)
            size_mb = os.path.getsize(full_path) / (1024 * 1024)
            print(f"  📦 {file} ({size_mb:.2f} MB)")
            mlmodels_found = True
    
    if not mlmodels_found:
        print("  ⚠️  No CoreML models found. Run the export cell first.")
else:
    print(f"❌ Weights directory not found: {weights_dir}")
    print("   Make sure training has completed successfully.")


In [ ]:
# Copy the CoreML model to Google Drive for easy download
from google.colab import drive
import shutil
import os

# Mount Google Drive if not already mounted
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Find and copy the CoreML model
weights_dir = "/content/runs/detect/train/weights/"

if os.path.exists(weights_dir):
    found = False
    for file in os.listdir(weights_dir):
        if file.endswith('.mlpackage'):
            source = os.path.join(weights_dir, file)
            dest = f"/content/drive/MyDrive/{file}"
            
            # Copy the entire .mlpackage directory
            if os.path.isdir(source):
                if os.path.exists(dest):
                    shutil.rmtree(dest)
                shutil.copytree(source, dest)
                print(f"✅ Copied {file} to Google Drive")
                print(f"📍 Location: MyDrive/{file}")
                found = True
            break
    
    if not found:
        print("❌ No .mlpackage file found. Make sure to run the export cell first.")
else:
    print(f"❌ Weights directory not found: {weights_dir}")
    print("   Make sure training and export completed successfully.")


In [ ]:
# Optional: Rename the model to match the expected name in Swift code
# The Swift code expects 'yolov11n.mlpackage'

import os
import shutil

# Find the exported CoreML model
weights_dir = "/content/runs/detect/train/weights/"
source_file = None

if os.path.exists(weights_dir):
    for file in os.listdir(weights_dir):
        if file.endswith('.mlpackage'):
            source_file = os.path.join(weights_dir, file)
            break
    
    if source_file and os.path.isdir(source_file):
        # Create a copy with the expected name
        dest_file = os.path.join(weights_dir, "yolov11n.mlpackage")
        
        if os.path.exists(dest_file):
            shutil.rmtree(dest_file)
        
        shutil.copytree(source_file, dest_file)
        print(f"✅ Created copy: yolov11n.mlpackage")
        print(f"📦 This file is ready to use in your Xcode project")
        
        # Also copy to Google Drive
        from google.colab import drive
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
        
        gdrive_dest = "/content/drive/MyDrive/yolov11n.mlpackage"
        if os.path.exists(gdrive_dest):
            shutil.rmtree(gdrive_dest)
        shutil.copytree(dest_file, gdrive_dest)
        print(f"✅ Also saved to Google Drive: MyDrive/yolov11n.mlpackage")
    else:
        print("❌ No .mlpackage file found. Make sure to run the export cell first.")
else:
    print(f"❌ Weights directory not found: {weights_dir}")
    print("   Make sure training and export completed successfully.")


# 📱 Using the Model in iOS/Swift

## Steps to integrate the CoreML model in your Xcode project:

1. **Download the model from Google Drive**
   - Download `yolov11n.mlpackage` from your Google Drive

2. **Add to Xcode project**
   - Drag `yolov11n.mlpackage` into your Xcode project navigator
   - Make sure "Copy items if needed" is checked
   - Ensure the file is added to your app target

3. **Verify the model type in Xcode**
   - Select `yolov11n.mlpackage` in Xcode
   - In the File Inspector (right panel), set **Type** to "Core ML Model"
   - Verify **Target Membership** is checked for your app

4. **Clean and rebuild**
   - In Xcode: Product → Clean Build Folder (⌘⇧K)
   - Build the project (⌘B)
   - The model will be compiled to `yolov11n.mlmodelc`

5. **Use in Swift code**
   - The `BusLabelDetectorView.swift` is already configured to load this model
   - It will automatically detect bus labels and extract text using OCR

## Model Information

- **Input**: Image (640x640 pixels, RGB)
- **Output**: Object detections with bounding boxes
- **Classes**: Bus labels (as defined in your training data)
- **Framework**: Vision + CoreML
- **NMS**: Included in the model for better performance
